<a href="https://www.kaggle.com/code/nadaaglan/australian-vehicle-prices-prediction?scriptVersionId=180242850" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/australian-vehicle-prices/Australian Vehicle Prices.csv


In [40]:
# reading dataset
data = pd.read_csv(
    '/kaggle/input/australian-vehicle-prices/Australian Vehicle Prices.csv',
    na_values=['POA', '-', '- / -'],
)

# For simplicity, we drop the columns with too many categorical values
data.drop(columns=['Brand', 'Model', 'Car/Suv', 'Title', 'ColourExtInt', 'Location'], inplace=True)

data.head()

,Year,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,CylindersinEngine,BodyType,Doors,Seats,Price
0,2022.0,DEMO,Automatic,"4 cyl, 2.2 L",AWD,Diesel,8.7 L / 100 km,5595.0,4 cyl,SUV,4 Doors,7 Seats,51990.0
1,2022.0,USED,Automatic,"4 cyl, 1.5 L",Front,Premium,6.7 L / 100 km,16.0,4 cyl,Hatchback,5 Doors,5 Seats,19990.0
2,2022.0,USED,Automatic,"4 cyl, 2 L",Rear,Premium,6.6 L / 100 km,8472.0,4 cyl,Coupe,2 Doors,4 Seats,108988.0
3,2011.0,USED,Automatic,"8 cyl, 5.5 L",Rear,Premium,11 L / 100 km,136517.0,8 cyl,Coupe,2 Doors,4 Seats,32990.0
4,2022.0,USED,Automatic,"4 cyl, 1.3 L",Front,Unleaded,6 L / 100 km,1035.0,4 cyl,SUV,4 Doors,5 Seats,34990.0


# Using Regular Expressions (Regex)
We use regex to extract numbers from the following columns: `Engine`, `FuelConsumption`, `CylindersinEngine`, `Doors`, and `Seats`.

Run the cells below.

In [41]:
# Helper function to extract numbers from string columns
def extract_number(data: 'pd.DataFrame', 
                  col: 'str', 
                  pattern: 'str') -> 'pd.Series':
    '''
    Extracts a number using the given regex pattern from the given data and col, 
    converts the result to float, and returns the result as a pd.Series.
    '''
    return data[col].str.extract(pattern, expand=False).astype('float32')

In [42]:
# We use the previous hepler function to extract numbers from string columns
data['Engine'] = extract_number(data, col='Engine', pattern=r'.*?(\d+\.?\d*) L')
data['FuelConsumption'] = extract_number(data, col='FuelConsumption', pattern=r'(\d+\.?\d*) L.*')
data['CylindersinEngine'] = extract_number(data, col='CylindersinEngine', pattern=r'(\d+) (?:cyl|L)')
data['Doors'] = extract_number(data, col='Doors', pattern=r'(\d+) Doors')
data['Seats'] = extract_number(data, col='Seats', pattern=r'(\d+) Seats')

data.head()

,Year,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,CylindersinEngine,BodyType,Doors,Seats,Price
0,2022.0,DEMO,Automatic,2.2,AWD,Diesel,8.7,5595.0,4.0,SUV,4.0,7.0,51990.0
1,2022.0,USED,Automatic,1.5,Front,Premium,6.7,16.0,4.0,Hatchback,5.0,5.0,19990.0
2,2022.0,USED,Automatic,2.0,Rear,Premium,6.6,8472.0,4.0,Coupe,2.0,4.0,108988.0
3,2011.0,USED,Automatic,5.5,Rear,Premium,11.0,136517.0,8.0,Coupe,2.0,4.0,32990.0
4,2022.0,USED,Automatic,1.3,Front,Unleaded,6.0,1035.0,4.0,SUV,4.0,5.0,34990.0


# Data Analysis
Examine the dataset and answer the following questions:
1. How many samples and features does this dataset contain?
2. What is the target column?
3. What is the data type of each column?
4. Which columns are numeric? Which columns are categorical?
5. Does this dataset contain missing values?

In [43]:
# Write your analysis code here
# Feel free to create as many cells as you want
#1-How many samples and features does this dataset contain?

feature_num = data.shape[1]
sample_num  =data.shape[0]
# features number represent the columns count, and samples number repreent the rows count.
print(f"Feature's number = {feature_num}")
print(f"Sample's number = {sample_num}")

Feature's number = 13
Sample's number = 16734


In [44]:
# 2_ What is the target column?

#since it's a price prediction dataset, the target column is "Price".

target_col = 'Price'
target_vals = data[target_col]

target_df = pd.DataFrame(data[target_col])
target_df


,Price
0,51990.0
1,19990.0
2,108988.0
3,32990.0
4,34990.0
...,...
16729,29500.0
16730,39888.0
16731,35280.0
16732,41888.0


In [45]:
# 3_What is the data type of each column?

data_types_df = pd.DataFrame(data.dtypes)
data_types_df

,0
Year,float64
UsedOrNew,object
Transmission,object
Engine,float32
DriveType,object
FuelType,object
FuelConsumption,float32
Kilometres,float64
CylindersinEngine,float32
BodyType,object


In [46]:
# 4_Which columns are numeric? Which columns are categorical?

numeric_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(include='object').columns

print("Numeric Columns:")
print(numeric_columns)


print("Categorical Columns:")
print(categorical_columns)

Numeric Columns:
Index(['Year', 'Engine', 'FuelConsumption', 'Kilometres', 'CylindersinEngine',
       'Doors', 'Seats', 'Price'],
      dtype='object')
Categorical Columns:
Index(['UsedOrNew', 'Transmission', 'DriveType', 'FuelType', 'BodyType'], dtype='object')


In [47]:
# 5_Does this dataset contain missing values?

mask = data.isnull().any(axis = 0)
mask

#from displaying the columns nan values, it turns out that YES there's a missing values:"

Year                 True
UsedOrNew            True
Transmission         True
Engine               True
DriveType            True
FuelType             True
FuelConsumption      True
Kilometres           True
CylindersinEngine    True
BodyType             True
Doors                True
Seats                True
Price                True
dtype: bool

# Data Cleaning

Remove missing values (if any)

In [48]:
# here we're deleting the rows containing NAN values
# the percentage of rows containing NAN values = 0.1464682681964862 (removing is valid)
mask = data.isnull().any(axis = 1)

nan_rows = mask.sum()
total_rows = len(data)

nan_rows/total_rows

clean_data = data[~mask]
clean_data

,Year,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,CylindersinEngine,BodyType,Doors,Seats,Price
0,2022.0,DEMO,Automatic,2.2,AWD,Diesel,8.7,5595.0,4.0,SUV,4.0,7.0,51990.0
1,2022.0,USED,Automatic,1.5,Front,Premium,6.7,16.0,4.0,Hatchback,5.0,5.0,19990.0
2,2022.0,USED,Automatic,2.0,Rear,Premium,6.6,8472.0,4.0,Coupe,2.0,4.0,108988.0
3,2011.0,USED,Automatic,5.5,Rear,Premium,11.0,136517.0,8.0,Coupe,2.0,4.0,32990.0
4,2022.0,USED,Automatic,1.3,Front,Unleaded,6.0,1035.0,4.0,SUV,4.0,5.0,34990.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16727,2015.0,USED,Automatic,1.6,Front,Unleaded,6.9,54359.0,4.0,Wagon,4.0,5.0,21450.0
16728,2021.0,USED,Automatic,2.0,Front,Unleaded,6.9,32977.0,4.0,SUV,4.0,5.0,36770.0
16731,2022.0,DEMO,Automatic,1.6,Front,Unleaded,6.2,5165.0,4.0,SUV,4.0,5.0,35280.0
16732,2016.0,USED,Automatic,2.0,AWD,Premium,7.2,85525.0,4.0,SUV,4.0,5.0,41888.0


In [49]:
#here we're representing the NAN values within each column  (feature removing ain't valid in this case)
data.isnull().sum()/len(data) 

Year                 0.000060
UsedOrNew            0.000060
Transmission         0.015059
Engine               0.100155
DriveType            0.000060
FuelType             0.038126
FuelConsumption      0.101470
Kilometres           0.035377
CylindersinEngine    0.100096
BodyType             0.016852
Doors                0.100096
Seats                0.101888
Price                0.003167
dtype: float64

# Data Encoding
Convert the categorical features to numeric

In [53]:
# Use a suitable encoding to convert categorical features to numeric

# we'll be using one-hot-encoding to convert categorical features to numeric.
data_hot_encoded= pd.get_dummies(clean_data)
data_hot_encoded.replace({True: 1, False: 0}, inplace=True)

data_hot_encoded.head()

,Year,Engine,FuelConsumption,Kilometres,CylindersinEngine,Doors,Seats,Price,UsedOrNew_DEMO,UsedOrNew_NEW,...,BodyType_Commercial,BodyType_Convertible,BodyType_Coupe,BodyType_Hatchback,BodyType_Other,BodyType_People Mover,BodyType_SUV,BodyType_Sedan,BodyType_Ute / Tray,BodyType_Wagon
0,2022.0,2.2,8.7,5595.0,4.0,4.0,7.0,51990.0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,2022.0,1.5,6.7,16.0,4.0,5.0,5.0,19990.0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2022.0,2.0,6.6,8472.0,4.0,2.0,4.0,108988.0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2011.0,5.5,11.0,136517.0,8.0,2.0,4.0,32990.0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2022.0,1.3,6.0,1035.0,4.0,4.0,5.0,34990.0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [54]:
data_hot_encoded.dtypes

Year                      float64
Engine                    float32
FuelConsumption           float32
Kilometres                float64
CylindersinEngine         float32
Doors                     float32
Seats                     float32
Price                     float64
UsedOrNew_DEMO              int64
UsedOrNew_NEW               int64
UsedOrNew_USED              int64
Transmission_Automatic      int64
Transmission_Manual         int64
DriveType_4WD               int64
DriveType_AWD               int64
DriveType_Front             int64
DriveType_Other             int64
DriveType_Rear              int64
FuelType_Diesel             int64
FuelType_Electric           int64
FuelType_Hybrid             int64
FuelType_LPG                int64
FuelType_Leaded             int64
FuelType_Premium            int64
FuelType_Unleaded           int64
BodyType_Commercial         int64
BodyType_Convertible        int64
BodyType_Coupe              int64
BodyType_Hatchback          int64
BodyType_Other

# Data Splitting
Split the dataset into
- Input and output
- Training and test sets

In [55]:
# Split the dataset to input and output

data_input = data_hot_encoded.drop(columns=['Price'])
data_output = data_hot_encoded['Price']

#data_input.head()
data_output.head()

0     51990.0
1     19990.0
2    108988.0
3     32990.0
4     34990.0
Name: Price, dtype: float64

In [58]:
# Split the input and output to training and test sets
from sklearn.model_selection import train_test_split

x,x_test,y,y_test= train_test_split(
    data_input, data_output, test_size=0.33,
    random_state=0)
x_train, x_val, y_train,y_val = train_test_split(
    x,y,test_size = 0.33, random_state = 0
)

In [59]:
print(x_train.shape)
print(y_train.shape)
print('------------------')
print(x_val.shape)
print(y_val.shape)
print('------------------')
print(x_test.shape)
print(y_test.shape)
print('------------------')

(6411, 34)
(6411,)
------------------
(3158, 34)
(3158,)
------------------
(4714, 34)
(4714,)
------------------


# Linear Regression

Train a linear regression model and calculate its $R^2$ (r2_score) for training and test sets

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [62]:
# Create a linear regression model
linear_reg = LinearRegression()

# Train the model
linear_reg.fit(x_train, y_train)

# Make predictions for training and test sets
y_pred_train = linear_reg.predict(x_train)
y_pred_val = linear_reg.predict(x_val)


# Calculate r2_score for training and val sets
print('train', r2_score(y_train, y_pred_train))
print('val', r2_score(y_val, y_pred_val))

train 0.5528565548088307
val 0.5375085745832817


In [64]:
#testing prediction
y_pred_test = linear_reg.predict(x_test)
print('test', r2_score(y_test, y_pred_test))


#DAMN, THE ACCURACY SUCKS

test 0.5371751279367274
